In [112]:
import pyspark.sql
import pyspark.sql.types as t
import pyspark.sql.functions as f

global spark

spark = (
    pyspark.sql.SparkSession
   .builder
   .master("local[*]")
   .getOrCreate()
)

In [94]:
associations = 'gwas_associations_2022-05-27.tsv'
studies = 'gwas_studies_new_2022-05-27.tsv'
studies_unpublished = 'gwas_unpublished_studies_2022-05-27.tsv'
ancestries = 'gwas_ancestries_new_2022-05-27.tsv'
ancestries_unpublished = 'gwas_unpublished_ancestries_2022-05-27.tsv'

# Read and combine the two ancesty files:
ancestries_df = (
    spark.read.csv(ancestries, sep='\t', header=True)
    .union(
        spark.read.csv(ancestries_unpublished, sep='\t', header=True)
    )
    .drop('PUBMED ID', 'FIRST AUTHOR', 'DATE')
    .persist()
)

# Read and combined the two study files:
studies_df = (
    spark.read.csv(studies, sep='\t', header=True)
    .union(
        spark.read.csv(studies_unpublished, sep='\t', header=True)
        .withColumnRenamed('MAPPED_TRAIT_URI', 'MAPPED TRAIT URI')
        .withColumnRenamed('MAPPED_TRAIT', 'MAPPED TRAIT')
    )
)

# Group by study id and calculate number:
joined = (
    studies_df
    .join(ancestries_df, how='outer', on='STUDY ACCESSION')
    .persist()
)

joined.count()

63052

In [95]:
DROPPED_STUDIES = [
    'GCST005806', # Sun et al pQTL study
    'GCST005837' # Huang et al IBD study
]

(
    joined
    .filter(~f.col('STUDY ACCESSION').isin(DROPPED_STUDIES))
    .count()
)

63050

In [96]:
ancestries_columns = ancestries_df.columns
study_columns = spark.read.csv(studies, sep='\t', header=True).columns

[col for col in ancestries_columns if col in study_columns]

['STUDY ACCESSION']

In [97]:
(
    ancestries_df
    .select('STUDY ACCESSION', 'INITIAL SAMPLE DESCRIPTION', 
            'REPLICATION SAMPLE DESCRIPTION', 'NUMBER OF INDIVIDUALS')
    .show()
)
# .show(1, False, True)

+---------------+--------------------------+------------------------------+---------------------+
|STUDY ACCESSION|INITIAL SAMPLE DESCRIPTION|REPLICATION SAMPLE DESCRIPTION|NUMBER OF INDIVIDUALS|
+---------------+--------------------------+------------------------------+---------------------+
|     GCST009498|      up to 182,902 Eur...|                            NA|               182902|
|     GCST009499|      up to 182,902 Eur...|                            NA|               182902|
|     GCST009500|      up to 182,902 Eur...|                            NA|               182902|
|     GCST009638|      5,321 European an...|          2,237 cases, 3,46...|                 5698|
|     GCST009638|      5,321 European an...|          2,237 cases, 3,46...|                21987|
|     GCST009637|      1,369 European an...|          422 cases, 3,461 ...|                 3883|
|     GCST009637|      1,369 European an...|          422 cases, 3,461 ...|                18035|
|     GCST009636|   

In [98]:
import re

@f.udf(t.StructType([
    t.StructField('n_initial', t.IntegerType()),
    t.StructField('n_cases', t.IntegerType())
]))
def extract_sample_sizes(s):
    ''' Extracts sample size info from GWAS Catalog
        "INITIAL SAMPLE SIZE" field
    Returns:
        total N, N cases
    '''
    n_cases = 0
    n_controls = 0
    n_quant = 0
    for part in s.split(', '):
        # Extract sample size
        mtch = re.search('([0-9,]+)', part)
        if mtch:
            n = int(mtch.group(1).replace(',', ''))
            # Add to correct counter
            if 'cases' in part:
                n_cases += n
            elif 'controls' in part:
                n_controls += n
            else:
                n_quant += n

    # Return n_total and n_cases
    if n_quant > 0:
        return n_quant, 0
    else:
        return n_cases + n_controls, n_cases



In [109]:
# Parsing samples from 
studies_w_samples = (
    studies_df
    .withColumn('parsedSamples', extract_sample_sizes(f.col('INITIAL SAMPLE SIZE')))
    .withColumn('n_cases', f.col('parsedSamples.cases'))
    .withColumn('n_initial', f.col('parsedSamples.sampleSize'))
    .select('STUDY ACCESSION', 'n_cases', 'n_initial')
    .persist()
)

# 43810 -> 43810
studies_w_samples.select('STUDY ACCESSION').distinct().count()

AnalysisException: No such struct field cases in n_initial, n_cases

In [100]:
# Parses sample sizes from ancestry:
studies_w_samples_ancestry = (
    ancestries_df
    .groupBy('STUDY ACCESSION')
    .agg(
        f.sum('NUMBER OF INDIVIDUALS').cast(t.IntegerType()).alias('n_initial_ancestry'),
        f.sum('NUMBER OF CASES').cast(t.IntegerType()).alias('n_cases_ancestry')
    )
    .persist()
)

studies_w_samples_ancestry.count()

43943

In [101]:
(
    studies_w_samples_ancestry
    .join(studies_w_samples, on='STUDY ACCESSION', how='outer')
    .show()
)

+---------------+------------------+----------------+-------+---------+
|STUDY ACCESSION|n_initial_ancestry|n_cases_ancestry|n_cases|n_initial|
+---------------+------------------+----------------+-------+---------+
|     GCST000101|              1094|            null|      0|     1094|
|     GCST000549|              2362|            null|   2362|     2362|
|     GCST000926|              1012|            null|      0|      470|
|     GCST000994|              7128|            null|    733|     1462|
|     GCST000996|              6530|            null|   1717|     6530|
|     GCST001229|              2813|            null|    765|     1639|
|     GCST001294|              1772|            null|      0|     1772|
|     GCST001303|             12347|            null|   2315|    12347|
|     GCST001526|             85573|            null|      0|    51750|
|     GCST001565|             30518|            null|   2111|     4646|
|     GCST001602|               282|            null|    141|   

In [102]:
(
    ancestries_df
    .filter(f.col('STUDY ACCESSION') == 'GCST000994')
    .show(2, False, True)
#     .count()
)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------
 STUDY ACCESSION                         | GCST000994                                                                                                      
 INITIAL SAMPLE DESCRIPTION              | 733 Japanese ancestry cases, 729 Japanese ancestry controls                                                     
 REPLICATION SAMPLE DESCRIPTION          | 2,794 Japanese ancestry drinkers, 1,521 Japanese ancestry chance drinkers, 1,351 Japanese ancestry non-drinkers 
 STAGE                                   | replication                                                                                                     
 NUMBER OF INDIVIDUALS                   | 5666                                                                                                            
 BROAD ANCESTRAL CATEGORY                | East Asian           

In [103]:
(
    studies_df
    .filter(f.col('STUDY ACCESSION') == 'GCST000994')
    .show(2, False, True)
)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG       | 2011-04-07                                                                                                                                         
 PUBMED ID                   | 21372407                                                                                                                                           
 FIRST AUTHOR                | Takeuchi F                                                                                                                                         
 DATE                        | 2011-03-01                                                                                                                                         
 JOURNAL                     | Circ J                                                                    

In [107]:
# There are discrepancies between the study lists...
# Studies in study table but not in ancestry:
len(
    studies_df
    .join(ancestries_df, on='STUDY ACCESSION', how='left_anti')
    .select('STUDY ACCESSION')
    .drop_duplicates()
    .toPandas()
    ['STUDY ACCESSION']
    .to_list()
)

15

In [108]:
(
    ancestries_df
    .show(1, False, True)
)

-RECORD 0------------------------------------------------------------------------------
 STUDY ACCESSION                         | GCST009498                                  
 INITIAL SAMPLE DESCRIPTION              | up to 182,902 European ancestry individuals 
 REPLICATION SAMPLE DESCRIPTION          | NA                                          
 STAGE                                   | initial                                     
 NUMBER OF INDIVIDUALS                   | 182902                                      
 BROAD ANCESTRAL CATEGORY                | European                                    
 COUNTRY OF ORIGIN                       | NR                                          
 COUNTRY OF RECRUITMENT                  | NR                                          
 ADDITIONAL ANCESTRY DESCRIPTION         | null                                        
 ANCESTRY DESCRIPTOR                     | null                                        
 FOUNDER/GENETICALLY ISOLATED PO

In [86]:
studies_df.select('SUMMARY STATS LOCATION').show()

+----------------------+
|SUMMARY STATS LOCATION|
+----------------------+
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
+----------------------+
only showing top 20 rows



In [80]:
import pandas as pd
aids = (
    pd.read_csv('gwas_unpublished_ancestries_2022-05-24.tsv', sep='\t')
    .loc[lambda df: df['STUDY ACCESSION'].isin(sudy_only)]
    ['STUDY ACCESSION']
    .drop_duplicates()
    .to_list()
)

[x for x in sudy_only if x not in aids]

['FEIRI and ASKLEPIOS case control study / MayoVDB case control study / DEFINE-FMD case control study / UM/CCF-FMD case control study"']

In [84]:
(
    pd.read_csv('gwas_studies_new_2022-05-24.tsv', sep='\t')
    .loc[lambda df: df['STUDY ACCESSION'].isin(ancestry_only)]
    ['STUDY ACCESSION']
    .drop_duplicates()
    .to_list()
)

['GCST000870', 'GCST000218', 'GCST000768']

In [110]:
%%bash


grep GCST009498 gwas_ancestries_new_2022-05-27.tsv

GCST009498	31681408	Yang XL	2019-10-11	up to 182,902 European ancestry individuals	NA	initial	182902	European	NR	NR							


In [4]:
import pandas as pd


# df = pd.read_csv('https://raw.githubusercontent.com/opentargets/curation/il-2604/gene_burden/curated_evidence.tsv', sep='\t')
df.literature.value_counts()

31981491    102
35568032     21
35396579      9
31085678      3
31327507      2
Name: literature, dtype: int64

In [33]:
import requests

data_type = 'ontology'
IMPC_SOLR_HOST = 'https://www.ebi.ac.uk/mi/impc/solr/phenodigm/select'
IMPC_SOLR_TIMEOUT = 3600

params = {'q': '*:*', 'fq': f'type:{data_type}', 'rows': 0}
response = requests.get(IMPC_SOLR_HOST, params=params, timeout=IMPC_SOLR_TIMEOUT)
response.raise_for_status()  # Check for HTTP errors. This will be caught by @retry.
response.json()['response']['numFound']

30762

In [25]:
params = {
    'q': '*:*',
    'group':'true', 
    'group.field': 'type', 
    'group.ngroups': 'true', 
    'fl': 'numFound,groupValue'
}
response = requests.get(IMPC_SOLR_HOST, params=params, timeout=IMPC_SOLR_TIMEOUT)
response.raise_for_status()  # Check for HTTP errors. This will be caught by @retry.
response.json()



{'responseHeader': {'status': 0,
  'QTime': 2336,
  'params': {'q': '*:*',
   'fl': 'numFound,groupValue',
   'group.ngroups': 'true',
   'group.field': 'type',
   'group': 'true'}},
 'grouped': {'type': {'matches': 8344853,
   'ngroups': 8,
   'groups': [{'groupValue': 'gene_gene',
     'doclist': {'numFound': 27370, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'ontology',
     'doclist': {'numFound': 30762, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'gene',
     'doclist': {'numFound': 351535, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'disease',
     'doclist': {'numFound': 20314, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'mouse_model',
     'doclist': {'numFound': 55706, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'disease_search',
     'doclist': {'numFound': 20314, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'disease_gene_summary',
     'doclist': {'numFound': 16204, 'start': 0, 'docs': [{}]}},
    {'groupValue': 'disease_model_summary',
     'doclist': {'num

In [53]:
import requests
from bs4 import BeautifulSoup

d = requests.get('http://188.226.200.126/random', headers={"Host": "bash.hu" }).text
soup = BeautifulSoup(d, 'html.parser', from_encoding='iso-8859-2')


/Users/dsuveges/opt/anaconda3/envs/jupyter3.9/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [61]:
print([23].text)


)KKAFITTER)Shpongy(FKL(MaNuAlDeep): bazz tegnap hazajöttem rohadtul betépve bemegyek a konyhába találtam egy rohadt nagy fürt szõlõt de bazz akkora volt egy darab szõlõszem mint egy körte...és oan furán csüngtek...és télleg kurva nagy volt...de bezasbáltam az összeset és oan kemény volt meg a magok is furák voltak....erre felkelek reggel és hallom hogy ordít anyám hogy: KI ETTE MEG AZ ÖSSZES KÖRTÉT???
baszki össze voltak fogva cérnával a körték:S
:[Fenszter:[: ez bashu keményenXD
		


In [63]:
# <a name="q45325" class="qanch">

ids = [name.get('name') for name in soup.findAll('a', class_='qanch')]
texts = [text.text for text in soup.findAll('div', class_='qtxt')]



In [7]:
%load_ext rpy2.ipython

In [93]:
%%R

library('jsonlite')
library('httr')

get_associations = function(target_id){
    query_url = 'https://api.platform.opentargets.org/api/v4/graphql'

    # Building query:
    request_body = list(
        operationName= 'TargetAssociationsQuery',
        variables = list(
            ensemblId= target_id,
            index= 0,
            size= 10000,
            sortBy= 'score',
            filter= '',
            aggregationFilters = list()

        ),
        query = '
            query TargetAssociationsQuery($ensemblId: String!, $index: Int!, $size: Int!, $filter: String, $sortBy: String!, $aggregationFilters: [AggregationFilter!]) {
                target(ensemblId: $ensemblId) {
                    id
                    approvedSymbol
                    approvedName
                    associatedDiseases(page: {index: $index, size: $size}, orderByScore: $sortBy, BFilter: $filter, aggregationFilters: $aggregationFilters) {
                        count
                        rows {
                            disease {
                                id
                                name
                            }
                            score
                            datatypeScores {
                                componentId: id
                                score
                            }
                        }
                    }
                }
            }
        '
    )

    # Retrieve data:
    response = POST(query_url, body=query, encode='json')

    # Parse data:
    char = rawToChar(response$content)
    data = jsonlite::fromJSON(char)

    # Extracting associations:
    associations = data$data$target$associatedDiseases$rows

    # Adding target and disease columns to dataframe:
    associations$targetSymbol = data$data$target$approvedSymbol
    associations$targetId = data$data$target$id
    associations$diseaseId = associations$disease$id
    associations$diseaseName = associations$disease$name

    # Dropping unused columns and return:
    return (associations[, c('targetId', 'targetSymbol', 'diseaseId', 'diseaseName', 'score', 'datatypeScores')])
}


target_id = 'ENSG00000065361'

get_associations(target_id)

           targetId targetSymbol       diseaseId
1   ENSG00000065361        ERBB3     EFO_0000311
2   ENSG00000065361        ERBB3 Orphanet_137776
3   ENSG00000065361        ERBB3     EFO_0000503
4   ENSG00000065361        ERBB3     EFO_0000616
5   ENSG00000065361        ERBB3   MONDO_0001187
6   ENSG00000065361        ERBB3   MONDO_0002108
7   ENSG00000065361        ERBB3     EFO_0000305
8   ENSG00000065361        ERBB3     EFO_0006544
9   ENSG00000065361        ERBB3   MONDO_0015277
10  ENSG00000065361        ERBB3     EFO_1000613
11  ENSG00000065361        ERBB3     EFO_0003060
12  ENSG00000065361        ERBB3     EFO_0000304
13  ENSG00000065361        ERBB3     EFO_0000365
14  ENSG00000065361        ERBB3   MONDO_0011962
15  ENSG00000065361        ERBB3     EFO_0002892
16  ENSG00000065361        ERBB3   MONDO_0004986
17  ENSG00000065361        ERBB3   MONDO_0007573
18  ENSG00000065361        ERBB3     EFO_0001663
19  ENSG00000065361        ERBB3     EFO_0003869
20  ENSG00000065361 

167 ENSG00000065361        ERBB3     EFO_1000125
168 ENSG00000065361        ERBB3     EFO_1000195
169 ENSG00000065361        ERBB3     EFO_1000446
170 ENSG00000065361        ERBB3     EFO_1000138
171 ENSG00000065361        ERBB3   MONDO_0002038
172 ENSG00000065361        ERBB3     EFO_1000367
173 ENSG00000065361        ERBB3   MONDO_0002973
174 ENSG00000065361        ERBB3     EFO_0008524
175 ENSG00000065361        ERBB3     EFO_1000297
176 ENSG00000065361        ERBB3     EFO_0001642
177 ENSG00000065361        ERBB3     EFO_1001967
178 ENSG00000065361        ERBB3     EFO_1000242
179 ENSG00000065361        ERBB3     EFO_0006719
180 ENSG00000065361        ERBB3   MONDO_0003624
181 ENSG00000065361        ERBB3   MONDO_0004331
182 ENSG00000065361        ERBB3   MONDO_0003594
183 ENSG00000065361        ERBB3     EFO_1000919
184 ENSG00000065361        ERBB3   MONDO_0002652
185 ENSG00000065361        ERBB3     EFO_1000040
186 ENSG00000065361        ERBB3   MONDO_0003865
187 ENSG00000065361 

334 ENSG00000065361        ERBB3     EFO_0000681
335 ENSG00000065361        ERBB3     EFO_0000432
336 ENSG00000065361        ERBB3     EFO_0000760
337 ENSG00000065361        ERBB3   Orphanet_2495
338 ENSG00000065361        ERBB3     EFO_0000196
339 ENSG00000065361        ERBB3     EFO_0001422
340 ENSG00000065361        ERBB3     EFO_0000186
341 ENSG00000065361        ERBB3     EFO_0000500
342 ENSG00000065361        ERBB3  Orphanet_45448
343 ENSG00000065361        ERBB3     EFO_0002617
344 ENSG00000065361        ERBB3     EFO_0008517
345 ENSG00000065361        ERBB3     EFO_0000341
346 ENSG00000065361        ERBB3     EFO_0000707
347 ENSG00000065361        ERBB3  Orphanet_98249
348 ENSG00000065361        ERBB3     EFO_0009637
349 ENSG00000065361        ERBB3     EFO_1002008
350 ENSG00000065361        ERBB3     EFO_0001061
351 ENSG00000065361        ERBB3   MONDO_0003060
352 ENSG00000065361        ERBB3     EFO_1001331
353 ENSG00000065361        ERBB3     EFO_1000028
354 ENSG00000065361 

501 ENSG00000065361        ERBB3    Orphanet_166
502 ENSG00000065361        ERBB3   MONDO_0024660
503 ENSG00000065361        ERBB3    Orphanet_233
504 ENSG00000065361        ERBB3     EFO_0002503
505 ENSG00000065361        ERBB3     EFO_1002000
506 ENSG00000065361        ERBB3   MONDO_0002616
507 ENSG00000065361        ERBB3     EFO_1000177
508 ENSG00000065361        ERBB3     EFO_0005950
509 ENSG00000065361        ERBB3   MONDO_0000386
510 ENSG00000065361        ERBB3   MONDO_0021582
511 ENSG00000065361        ERBB3 Orphanet_275864
512 ENSG00000065361        ERBB3      HP_0001943
513 ENSG00000065361        ERBB3   MONDO_0005232
514 ENSG00000065361        ERBB3   MONDO_0018648
515 ENSG00000065361        ERBB3     EFO_0003063
516 ENSG00000065361        ERBB3      HP_0001395
517 ENSG00000065361        ERBB3     EFO_0005230
518 ENSG00000065361        ERBB3     EFO_0009322
519 ENSG00000065361        ERBB3   Orphanet_1465
520 ENSG00000065361        ERBB3   MONDO_0003036
521 ENSG00000065361 

58                                                    anaplastic oligodendroglioma
59                                                      clear cell renal carcinoma
60                                                     papillary thyroid carcinoma
61                                                pancreatic ductal adenocarcinoma
62                                        gastroesophageal junction adenocarcinoma
63                                                       alveolar rhabdomyosarcoma
64                                                    oral squamous cell carcinoma
65                                                        type 1 diabetes mellitus
66                                                      Digestive System Carcinoma
67                                                   ovarian serous adenocarcinoma
68                                                                  lung carcinoma
69                                                      Ampulla of Vater Carcinoma
70  

...

353                                                                                                                                          rna_expression, 0.0996298662753917
354                                                                                                                                              literature, 0.0972689276178594
355                                                                                                                                          rna_expression, 0.0923898023623228
356                                                                                                                                              literature, 0.0912014405183634
357                                                                                                                                              literature, 0.0911896196417432
358                                                                                                                     

399                                                                                                                                              literature, 0.0530250751250136
400                                                                                                                                              literature, 0.0526570187099422
401                                                                                                                                              literature, 0.0499854211369555
402                                                                                                                                              literature, 0.0491410728069394
403                                                                                                                                              literature, 0.0486344638089297
404                                                                                                                     

445                                                                                                                                              literature, 0.0303965398805811
446                                                                                                                                              literature, 0.0303965398805811
447                                                                                                                                              literature, 0.0303965398805811
448                                                                                                                                              literature, 0.0303965398805811
449                                                                                                                                              literature, 0.0303965398805811
450                                                                                                                     

491                                                                                                                                              literature, 0.0243172319044648
492                                                                                                                                              literature, 0.0243172319044648
493                                                                                                                                              literature, 0.0241483622384616
494                                                                                                                                              literature, 0.0227974049104358
495                                                                                                                                              literature, 0.0227974049104358
496                                                                                                                     

537                                                                                                                                              literature, 0.0121586159522324
538                                                                                                                                              literature, 0.0121586159522324
539                                                                                                                                              literature, 0.0121586159522324
540                                                                                                                                              literature, 0.0121586159522324
541                                                                                                                                              literature, 0.0121586159522324
542                                                                                                                     

In [92]:
%%R 


# names(associations)

NULL


In [71]:
%%R

class(associations$datatypeScores)

[1] "list"
